In [8]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader
import sys
from pathlib import Path
import os

In [29]:
output_name = "context"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [9]:
template_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts-templates"))
template_dir

'/root/graph-visualizer/prompts-templates'

In [10]:
output_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","model-results"))
output_dir

'/root/graph-visualizer/model-results'

In [11]:
template_filename = "story-prompt-with-ontology-v3.txt"

In [13]:
bedrock_runtime= boto3.client("bedrock-runtime",region_name="us-east-1")

In [14]:
environment = Environment(loader=FileSystemLoader(template_dir))
template = environment.get_template(template_filename)

In [15]:
prompt = template.render()

In [19]:
message={"role": "user","content": prompt}

In [20]:
messages = [message]

In [26]:
body=json.dumps(
{
    'messages':messages,
    'anthropic_version':'bedrock-2023-05-31',
    'top_p':0.5,
    'temperature':0.5,
    'top_k':5,
    'max_tokens':3000
})

In [27]:
response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType='application/json',
    accept='application/json',
    body=body
)

In [30]:
response_body = json.loads(response['body'].read())

In [31]:
response_body

{'id': 'msg_018pGVoSV5m5K2qe5dLAZxQb',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': '```json\n{\n    "nodes": [\n        {\n            "id": "Knowledge",\n            "attributes": "",\n            "type": "concept",\n            "group": 1\n        },\n        {\n            "id": "Monkey",\n            "attributes": "hasName, hasPersonalityTrait",\n            "type": "animal",\n            "group": 2\n        },\n        {\n            "id": "BerryTree",\n            "attributes": "isLocatedIn, isEdible",\n            "type": "plant",\n            "group": 3\n        },\n        {\n            "id": "RiverBank",\n            "attributes": "hasDescription, isReal",\n            "type": "location",\n            "group": 4\n        },\n        {\n            "id": "Crocodile",\n            "attributes": "hasName, hasSpecies, hasEmotionalState",\n            "type": "animal",\n            "group": 5\n        },\n        {\n            "id": "Berrie

In [32]:
full_output_name = f"{model_id}-{template_filename.split('.txt')[0]}-{output_name}"

In [34]:
same_name_count = int(len([f for f in os.listdir(output_dir) if full_output_name in f])/2)

In [35]:
file_count = "" if same_name_count ==0 else f'-{same_name_count}'

In [36]:
with open(os.path.join(output_dir,f"{full_output_name}{file_count}.json"),"w+") as f:
    json.dump(response_body,f)

In [39]:
with open(os.path.join(output_dir,f"{full_output_name}-graph{file_count}.json"),"w+") as f:
    json.dump(json.loads(response_body["content"][0]["text"].split("```")[1].split("json")[1]),f)